<a href="https://colab.research.google.com/github/pmontman/tmp_choicemodels/blob/main/nb/tutorials/WK_08_tuto_panel_mixed_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 8: Continuing with the panel ataPanel data and mixed logit

We will continue with the tomato sauce dataset, showcasing how to create random effects on the coefficients, as opposed to the 'agent' effect (random alternative specific constants).




We have two famous brands of catsup and 3 different package sizes.

A description of the dataset can be found [here](https://www.tandfonline.com/doi/pdf/10.1080/07350015.1994.10524547?casa_token=r4LpjVvgDW4AAAAA:FVG8mEexsQ37tJ2bvk7oxZZ9K_jvvMJ2WxglLzBaHQD0_0REkXmKGsPPxXw_LRGwN3YHY8-L-k8U)

# Description of the dataset

* **id**: household identifiers,
* **choice**: one of heinz41, heinz32, heinz28, hunts32.
* **disp_x**: is there a display for brand X ?
* **feat_x**: is there a newspaper feature advertisement for brand x?
* **price_x**: price of brand x

---
---

# Preparing the environment
*The preparation and dataset loading code is given to the students*

In [1]:
!pip install biogeme

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Load the packages, feel free to change the names.

In [2]:
import pandas  as pd
import numpy as np
import matplotlib.pyplot as plt

import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.expressions as exp
import biogeme.tools as tools
import biogeme.distributions as dist

# Load the dataset

In [3]:
path = 'https://raw.githubusercontent.com/pmontman/pub-choicemodels/main/data/catsup.csv'
catsup_pd = pd.read_csv(path)


In this case, notice the id variable, that identifies each household, so we have data from many choice situations for each household. We have also different amount of observations per household.

In [4]:
catsup_pd.head(7)

,id,disp_heinz41,disp_heinz32,disp_heinz28,disp_hunts32,feat_heinz41,feat_heinz32,feat_heinz28,feat_hunts32,price_heinz41,price_heinz32,price_heinz28,price_hunts32,choice
0,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz28
1,1,0,0,0,0,0,0,0,0,4.6,4.3,5.2,4.4,heinz28
2,1,0,0,0,0,0,1,0,0,4.6,2.5,4.6,4.8,heinz28
3,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz28
4,1,0,0,0,0,0,0,1,0,4.6,3.0,4.6,4.8,heinz28
5,1,0,0,0,0,0,0,0,0,5.0,3.0,4.7,3.0,heinz28
6,1,0,0,0,1,0,0,0,1,5.1,3.1,4.6,4.1,heinz28


# Auxiliary function

In [5]:
def qbus_update_globals_bgm(pd_df):
   globals().update(db.Database('tmp_bg_bgm_for_glob', pd_df).variables)

# Data cleaning: Preparing the dataset for Biogeme

Encode the choice variable (a string) into numbers with the `factorize` function.
We take a look at the codetable to know how the numbers are mapped to the alternatives. The order of the codetable indicates the association,
0: heinz28, 1: heinz41 and so on.

In [6]:
catsup_pd['choice'], codetable = catsup_pd['choice'].factorize()

In [7]:
codetable

Index(['heinz28', 'heinz41', 'heinz32', 'hunts32'], dtype='object')

In [8]:
catsup_pd['choice']

0       0
1       0
2       0
3       0
4       0
       ..
2793    1
2794    1
2795    3
2796    0
2797    0
Name: choice, Length: 2798, dtype: int64

#Capturing agent effect through mixed logit

In [9]:

# Define level of verbosity
import biogeme.messaging as msg
logger = msg.bioMessage()
logger.setSilent()


We will have to use some additional functionalities of biogeme so the auxiliary functions cannot be used. So we have to create the biogeme database manually.

In [10]:
database = db.Database("catsup", catsup_pd)

We now declare the coefficients in our model, we will make a simple model,
just the alternative-specif constants and the variables disp, feat and price.

In [11]:
ASC_heinz41 = exp.Beta('ASC_heinz41',0,None,None,0)
ASC_heinz32  = exp.Beta('ASC_heinz32 ',0,None,None,0)
ASC_heinz28 = exp.Beta('ASC_heinz28',0,None,None,0)
ASC_hunts32 = exp.Beta('ASC_hunts32',0,None,None,1)

B_disp = exp.Beta('B_disp',0,None,None,0)
B_feat = exp.Beta('B_feat',0,None,None,0)
B_price = exp.Beta('B_price',0,None,None,0)

We creat the parameters that will be the standard deviations

In [12]:

SIGMA_price = exp.Beta('SIGMA_price',1,0,None,0)
EC_price = SIGMA_price * exp.bioDraws('EC_price','NORMAL')


And the following code is how we tell biogeme that the parameters are random.
The `EC_`s are the agent effects. EC stands for Error Component. The key function that indicates randomness is the biogeme function `exp.bioDraws`, that indicates that they are drawn from a probability distribution. The second argument specifies the distribution, some possible values are `'NORMAL'` `'UNIFORMSYM'`.

In [13]:
#B_price_rnd = B_price + 

An this is how the specification of the utility functions looks like in the end.
The Betas (disp, feat and price) and ASCsare fixed effect, while the EC are random.

In [14]:
globals().update(database.variables)
# Definition of the utility functions
V_heinz41 = ASC_heinz41 + B_disp *disp_heinz41 + (B_feat) * feat_heinz41 + (B_price + EC_price) * price_heinz41 
V_heinz32 = ASC_heinz32 + B_disp *disp_heinz32 + B_feat * feat_heinz32 + (B_price  + EC_price) * price_heinz32 
V_heinz28 = ASC_heinz28 + B_disp *disp_heinz28 + B_feat * feat_heinz28 + (B_price + EC_price) * price_heinz28 
V_hunts32 = ASC_hunts32 + B_disp *disp_hunts32 + B_feat * feat_hunts32 + (B_price + EC_price) * price_hunts32 

We create the dictionary that maps to the alternatives. **Remember to be careful here.** The numbers should match the alternatives as we indicated by the factorize() transformation at the beginning of the notebook.
Availabilities are not considered here, we set them to 1.
Finaly, we specify the logit model, as usual.
These steps are common to the multinomial logit.

In [15]:
# Associate utility functions with the numbering of alternatives
V = {0: V_heinz28,
     1: V_heinz41,
     2: V_heinz32,
     3: V_hunts32}

av = {0: 1,
     1: 1,
     2: 1,
     3: 1}

# Conditional to the random variables, the likelihood of one observation is
# given by the logit model (called the kernel)
obsprob = models.logit(V,av, choice)


The difference from the usual declaration of the MNL comes now.

We have to do two new steps:

1. Tell biogeme to consider the panel nature of the data.
2. Tell biogeme to calculate the choice probabilities by simulation. This is how we deal with the random parameters. We simulate for the distribution, and the we calculate the likelihood for that distribution.

And Step 2 we take the model and the modifyi it by the expresion `exp.MonteCarlo`. The final log it to take the loglikelihood.

In [16]:
logprob = exp.log(exp.MonteCarlo(obsprob))

We we are using simulation, we have to tell biogeme how many draws from the distribution are we going to generate. The more draws, the more accurate estimation, but it is compuationally costly.

We also set up a seed, so we can get the same results if the run the notebook again (setting up a seed is a good habit in general)

In [17]:

# Create the Biogeme object
biogeme  = bio.BIOGEME(database,logprob,numberOfDraws=100, seed=1)




Estimation and results as usual.

In [18]:

# Estimate the parameters. 
results = biogeme.estimate()

We take a look at the results,

In [19]:
results.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_heinz28,2.447719,0.100295,24.405280,0.000000e+00
ASC_heinz32,1.518159,0.073230,20.731406,0.000000e+00
ASC_heinz41,1.370617,0.120309,11.392508,0.000000e+00
B_disp,0.887912,0.106160,8.363890,0.000000e+00
B_feat,0.920067,0.125651,7.322431,2.435829e-13
B_price,-1.436162,0.092175,-15.580874,0.000000e+00
SIGMA_price,0.283032,0.310318,0.912071,3.617313e-01


# Analyzing the random paremeters

So have recovered that the $\beta_{price}$ is a random variable with mean -1.47 and standard deviation $\sigma$ 0.40.
We can 'ask questions' to interpret what this actually means. The type of questions will ask us to calculate something about 



In [20]:
from scipy.stats import norm

mean_B_price = results.getBetaValues()['B_price']
sd_B_price = results.getBetaValues()['SIGMA_price']
mean_B_price, sd_B_price

(-1.4361624039031133, 0.28303223287403495)

#What is the proportion of people in the population that have a positive utility relationship with price?
Even though the mean of the $\beta_{price}$ is negative, because it is random it might happen that *some* people in the population will have a positive value for their $B_{price}$. We can calculate this easily using the normal cumulative distribution function (CDF).
Remember th CDF(x) is is the probability that values for that distribution is < x.
In our case, we want to calculate the prob. that a gaussian with mean -1.47 and sd 0.4 is smaller than 0.

In [21]:
norm.cdf(   0, loc=mean_B_price, scale=sd_B_price)

0.999999805436089

So 0.99984 are negative, the proportion that have positive values for $B_{price}$ is

In [22]:
1- norm.cdf(   0, loc=mean_B_price, scale=sd_B_price)

1.9456391098238157e-07

About 1 per 10000 in our population prefers to buy more expensive tomato sauces, all other things being equal.

# Comparing mixed specificaion vs the fixed effects specification

In [36]:
V_heinz41 = ASC_heinz41 + B_disp *disp_heinz41 + B_feat * feat_heinz41 + B_price * price_heinz41 
V_heinz32 = ASC_heinz32 + B_disp *disp_heinz32 + B_feat * feat_heinz32 + B_price  * price_heinz32 
V_heinz28 = ASC_heinz28 + B_disp *disp_heinz28 + B_feat * feat_heinz28 + B_price * price_heinz28 
V_hunts32 = ASC_hunts32 + B_disp *disp_hunts32 + B_feat * feat_hunts32 + B_price  * price_hunts32 

In [37]:
# Associate utility functions with the numbering of alternatives
V = {0: V_heinz28,
     1: V_heinz41,
     2: V_heinz32,
     3: V_hunts32}

av = {0: 1,
     1: 1,
     2: 1,
     3: 1}

# Conditional to the random variables, the likelihood of one observation is
# given by the logit model (called the kernel)
obsprob = models.loglogit(V,av, choice)


In [38]:
biogeme_fixed  = bio.BIOGEME(database,obsprob)
results_fixed = biogeme_fixed.estimate()
results_fixed.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_heinz28,2.425955,0.090695,26.748585,0.000000e+00
ASC_heinz32,1.501228,0.063947,23.476127,0.000000e+00
ASC_heinz41,1.353701,0.114962,11.775184,0.000000e+00
B_disp,0.875603,0.102257,8.562782,0.000000e+00
B_feat,0.908579,0.120177,7.560374,4.019007e-14
B_price,-1.402411,0.056095,-25.000760,0.000000e+00


In [39]:
def qbus_likeli_ratio_test_bgm(results_complex, results_reference, signif_level):
  return tools.likelihood_ratio_test( (results_complex.data.logLike, results_complex.data.nparam),
                                     (results_reference.data.logLike, results_reference.data.nparam), signif_level)

In [40]:
results.getGeneralStatistics()    

{'Number of estimated parameters': GeneralStatistic(value=7, format=''),
 'Sample size': GeneralStatistic(value=2798, format=''),
 'Excluded observations': GeneralStatistic(value=0, format=''),
 'Init log likelihood': GeneralStatistic(value=-6018.19697891139, format='.7g'),
 'Final log likelihood': GeneralStatistic(value=-2517.6554602899787, format='.7g'),
 'Likelihood ratio test for the init. model': GeneralStatistic(value=7001.083037242823, format='.7g'),
 'Rho-square for the init. model': GeneralStatistic(value=0.5816595121242794, format='.3g'),
 'Rho-square-bar for the init. model': GeneralStatistic(value=0.5804963730604487, format='.3g'),
 'Akaike Information Criterion': GeneralStatistic(value=5049.310920579957, format='.7g'),
 'Bayesian Information Criterion': GeneralStatistic(value=5090.8675416665355, format='.7g'),
 'Final gradient norm': GeneralStatistic(value=0.014646393343248675, format='.4E'),
 'Number of draws': GeneralStatistic(value=100, format=''),
 'Draws generation ti

In [41]:
results_fixed.getGeneralStatistics()  

{'Number of estimated parameters': GeneralStatistic(value=6, format=''),
 'Sample size': GeneralStatistic(value=2798, format=''),
 'Excluded observations': GeneralStatistic(value=0, format=''),
 'Init log likelihood': GeneralStatistic(value=-3093.6519669271274, format='.7g'),
 'Final log likelihood': GeneralStatistic(value=-2517.877249720801, format='.7g'),
 'Likelihood ratio test for the init. model': GeneralStatistic(value=1151.5494344126528, format='.7g'),
 'Rho-square for the init. model': GeneralStatistic(value=0.18611489700899797, format='.3g'),
 'Rho-square-bar for the init. model': GeneralStatistic(value=0.18417544161319288, format='.3g'),
 'Akaike Information Criterion': GeneralStatistic(value=5047.754499441602, format='.7g'),
 'Bayesian Information Criterion': GeneralStatistic(value=5083.374460372955, format='.7g'),
 'Final gradient norm': GeneralStatistic(value=0.0074695674779374395, format='.4E'),
 'Nbr of threads': GeneralStatistic(value=2, format='')}

In [42]:
qbus_likeli_ratio_test_bgm(results, results_fixed, 0.05)

LRTuple(message='H0 cannot be rejected at level 5.0%', statistic=0.44357886164470983, threshold=3.841458820694124)

# Panel data: Advanced stuff (not implemented)

We have seen that we can specify which variable has the 'panel' or 'grouping' information, for example, we can use the `id` variable to 

#Exercise: What is the proportion of households that react negatively to diplaying the catsup product?